# GSU Data Challenge 

Authors:

| Nom | Prenom | ID |
| -------- | ----- | -------- |
| Yurdakul | Bengü | 19401851 |
| Şengül | Beste | 19401831 |
| Erdi | Furkan | 21401966 |

## 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from ydata_profiling import ProfileReport

C:\Users\furka\AppData\Local\Temp\ipykernel_28768\1194981047.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',1000)

## 2. Importing Data

In [3]:
file_path = 'data.xlsx'

df = pd.read_excel(file_path)

df.head()

,Tarih,ÜrünKodu,FaturaNo,Birim,Miktar,KategoriAdı,AltKategoriAdi,MagazaKodu
0,2023-01-13,30.126.0072,331010041,ADET,1.0,Kahve Grubu,Açik Içecekler,C3
1,2023-01-13,30.126.0072,331020039,ADET,1.0,Kahve Grubu,Açik Içecekler,C3
2,2023-01-15,30.126.0072,331010047,ADET,1.0,Kahve Grubu,Açik Içecekler,C3
3,2023-01-18,30.126.0072,331020049,ADET,1.0,Kahve Grubu,Açik Içecekler,C3
4,2023-01-20,30.126.0072,331020055,ADET,1.0,Kahve Grubu,Açik Içecekler,C3


## 3. Data Preprocessing

In [4]:
# Filter rows where 'Miktar' is greater than 0
df = df[df['Miktar'] > 0]

replacements_for_kategori_adi = {
    'Alkollü İçecekler Grubu': 'Alkollü İçecek Grubu',
    'Alkolsüz İçecekler Grubu': 'Alkolsüz İçecek Grubu',
    'TA_Ticari Market Ürün Grubu': 'TA-Ticari Market Ürün Grubu'
}

# Replace the values in the 'KategoriAdı' column
df['KategoriAdı'] = df['KategoriAdı'].replace(replacements_for_kategori_adi)

# Define the replacements
replacements_for_alt_kategori_adi = {
    'Açik Içecekler': 'Açık Içecekler',
    'Balık Salata': 'Balıklı Salata',
    'Kasaönü': 'Kasa Önü',
    'Sandviç': 'Sandviçler'
}

# Replace the values in the 'AltKategoriAdi' column
df['AltKategoriAdi'] = df['AltKategoriAdi'].replace(replacements_for_alt_kategori_adi)

# Convert 'Tarih' column to datetime
df['Tarih'] = pd.to_datetime(df['Tarih'])

# Convert 'FaturaNo' column to integer
df['FaturaNo'] = df['FaturaNo'].astype(int)

# Convert 'Miktar' column to float
df['Miktar'] = df['Miktar'].astype(float)

# Create mappings for 'Birim', 'KategoriAdı', 'AltKategoriAdi', and 'MagazaKodu'
birim_mapping = {'ADET': 1, 'PORS': 2, 'DILIM': 3}
kategori_mapping = {kategori: i+1 for i, kategori in enumerate(df['KategoriAdı'].unique())}
alt_kategori_mapping = {alt_kategori: i+1 for i, alt_kategori in enumerate(df['AltKategoriAdi'].unique())}
magaza_mapping = {'A1': 1, 'B1': 2, 'B2': 3, 'C2': 4, 'C3': 5, 'D3': 6}

# Create new dataframes for mappings
birim_df = pd.DataFrame(list(birim_mapping.items()), columns=['Birim', 'Birim_ID'])
kategori_df = pd.DataFrame(list(kategori_mapping.items()), columns=['KategoriAdı', 'Kategori_ID'])
alt_kategori_df = pd.DataFrame(list(alt_kategori_mapping.items()), columns=['AltKategoriAdi', 'AltKategori_ID'])
magaza_df = pd.DataFrame(list(magaza_mapping.items()), columns=['MagazaKodu', 'Magaza_ID'])

birim_df.to_pickle("birim.pkl")
kategori_df.to_pickle("kategori.pkl")
alt_kategori_df.to_pickle("alt_kategori.pkl")
magaza_df.to_pickle("magaza.pkl")

# Apply mappings to respective columns
df['Birim'] = df['Birim'].map(birim_mapping)
df['KategoriAdı'] = df['KategoriAdı'].map(kategori_mapping)
df['AltKategoriAdi'] = df['AltKategoriAdi'].map(alt_kategori_mapping)
df['MagazaKodu'] = df['MagazaKodu'].map(magaza_mapping)

# Sum the faturas by other values 
df = df.groupby(df.columns.difference(['FaturaNo']).tolist(), as_index=False).agg({'Miktar': 'sum'})

# Sort the dataframe by Tarih and ÜrünKodu
df = df.sort_values(by=['Tarih', 'ÜrünKodu'])

In [5]:
df.to_pickle("preprocessed_data.pkl")

df

,AltKategoriAdi,Birim,KategoriAdı,MagazaKodu,Tarih,ÜrünKodu,Miktar
167229,51,1,22,1,2023-01-01,30.101.0079,1.0
167230,51,1,22,1,2023-01-01,30.101.0301,1.0
169961,51,2,22,1,2023-01-01,30.101.0540,1.0
347594,106,2,18,1,2023-01-01,30.101.0604,1.0
380825,116,2,2,2,2023-01-01,30.101.0605,2.0
...,...,...,...,...,...,...,...
184740,61,1,16,6,2023-11-30,30.304.0197,1.0
101884,30,1,16,4,2023-11-30,30.305.0102,2.0
102583,30,1,16,4,2023-11-30,30.305.0117,3.0
101885,30,1,16,4,2023-11-30,30.305.0118,1.0


## 4. Analyizing

In [6]:
profile = ProfileReport(df, title="GSU Data Challenge - TAV Data Profiling Report")

profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 382929 entries, 167229 to 239337
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   AltKategoriAdi  382929 non-null  int64         
 1   Birim           382929 non-null  int64         
 2   KategoriAdı     382929 non-null  int64         
 3   MagazaKodu      382929 non-null  int64         
 4   Tarih           382929 non-null  datetime64[ns]
 5   ÜrünKodu        382929 non-null  object        
 6   Miktar          382929 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 31.4+ MB


In [8]:
df.describe()

,AltKategoriAdi,Birim,KategoriAdı,MagazaKodu,Tarih,Miktar
count,382929.000000,382929.000000,382929.000000,382929.000000,382929,382929.000000
mean,60.416477,1.163255,13.168956,3.123694,2023-06-27 06:44:11.990317056,5.049798
min,1.000000,1.000000,1.000000,1.000000,2023-01-01 00:00:00,0.100000
25%,25.000000,1.000000,7.000000,2.000000,2023-04-18 00:00:00,2.000000
50%,66.000000,1.000000,15.000000,3.000000,2023-07-04 00:00:00,3.000000
75%,91.000000,1.000000,18.000000,4.000000,2023-09-10 00:00:00,5.000000
max,116.000000,3.000000,30.000000,6.000000,2023-11-30 00:00:00,619.000000
std,34.081895,0.400935,7.714357,1.512156,NaN,13.448828


In [9]:
# Group by MagazaKodu and iterate over each group
for magaza_kodu, group_df in df.groupby('MagazaKodu'):
    print(f"Summary statistics for Magaza {magaza_df.loc[magaza_df['Magaza_ID'] == magaza_kodu, 'MagazaKodu'].iloc[0]}:\n")
    print(group_df.head())
    print()
    print(group_df.describe())
    print()
    print("##############################################################################################################")

Summary statistics for Magaza A1:

        AltKategoriAdi  Birim  KategoriAdı  MagazaKodu      Tarih     ÜrünKodu  Miktar
167229              51      1           22           1 2023-01-01  30.101.0079     1.0
167230              51      1           22           1 2023-01-01  30.101.0301     1.0
169961              51      2           22           1 2023-01-01  30.101.0540     1.0
347594             106      2           18           1 2023-01-01  30.101.0604     1.0
170226              52      2            2           1 2023-01-01  30.101.0606     1.0

       AltKategoriAdi         Birim   KategoriAdı  MagazaKodu                          Tarih        Miktar
count    71401.000000  71401.000000  71401.000000     71401.0                          71401  71401.000000
mean        49.349519      1.207000     13.872845         1.0  2023-06-14 16:32:57.652974336      3.708161
min          2.000000      1.000000      1.000000         1.0            2023-01-01 00:00:00      0.200000
25%         17

## 5. Splitting Data to Test and Train the Model

In [28]:
# Sort the DataFrame by 'Tarih' and then by 'ÜrünKodu'
df_sorted = df.sort_values(by=['Tarih'])

split_date = "2023-10-30"

# Split the DataFrame into train and test sets
train_df = df_sorted[df_sorted['Tarih'] <= split_date]
train_df = train_df.sort_values(by=['Tarih', "ÜrünKodu"])

test_df = df_sorted[df_sorted['Tarih'] > split_date]
test_df = test_df.sort_values(by=['Tarih', "ÜrünKodu"])

train_df.to_pickle("train_data.pkl")
test_df.to_pickle("test_data.pkl")

# Display the shapes of train and test DataFrames
print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", test_df.shape)

Train DataFrame shape: (351324, 7)
Test DataFrame shape: (31605, 7)
